# Backpropagation Neural Network Implementation using NumPy

In [1]:
import numpy as np

# Define the activation function and its derivative
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

# Mean Squared Error Loss function
def mse_loss(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

# R² accuracy calculation (for regression problems)
def accuracy(y_true, y_pred):
    ss_res = np.sum((y_true - y_pred) ** 2)
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
    return 1 - (ss_res / ss_tot)

In [2]:
# Define the Backpropagation Neural Network class
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size, learning_rate):
        # Initialize network parameters
        self.learning_rate = learning_rate
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        # Initialize weights with random values
        self.weights_input_hidden = np.random.randn(input_size, hidden_size)
        self.weights_hidden_output = np.random.randn(hidden_size, output_size)
        
        # Initialize biases with random values
        self.bias_hidden = np.random.randn(hidden_size)
        self.bias_output = np.random.randn(output_size)

    def feedforward(self, X):
        # Feedforward through the network
        self.hidden_input = np.dot(X, self.weights_input_hidden) + self.bias_hidden
        self.hidden_output = sigmoid(self.hidden_input)
        
        self.final_input = np.dot(self.hidden_output, self.weights_hidden_output) + self.bias_output
        self.final_output = sigmoid(self.final_input)
        
        return self.final_output

    def backpropagate(self, X, y, output):
            # Compute error in the output
            output_error = y - output
            output_delta = output_error * sigmoid_derivative(output)
            
            # Compute error in the hidden layer
            hidden_error = np.dot(output_delta, self.weights_hidden_output.T)
            hidden_delta = hidden_error * sigmoid_derivative(self.hidden_output)
            
            # Update weights and biases (Gradient Descent)
            self.weights_hidden_output += np.dot(self.hidden_output.T, output_delta) * self.learning_rate
            self.weights_input_hidden += np.dot(X.T, hidden_delta) * self.learning_rate
            
            self.bias_output += np.sum(output_delta, axis=0) * self.learning_rate
            self.bias_hidden += np.sum(hidden_delta, axis=0) * self.learning_rate

    def train(self, X, y, epochs=1000):
        for epoch in range(epochs):
            output = self.feedforward(X)
            self.backpropagate(X, y, output)
            if epoch % 100 == 0:
                loss = mse_loss(y, output)
                print(f'Epoch {epoch}, Loss: {loss}')

    def predict(self, X):
        return self.feedforward(X)

In [3]:
import pandas as pd

# Preprocess the housing dataset for training
def preprocess_data(data):
    # Normalize the input features
    X = data[['RM', 'LSTAT', 'PTRATIO']].values
    X = (X - X.mean(axis=0)) / X.std(axis=0)
    
    # Normalize the target (MEDV)
    y = data[['MEDV']].values
    y = (y - y.min()) / (y.max() - y.min())
    
    return X, y

# Load the dataset
housing_data = pd.read_csv('housing.csv')
X, y = preprocess_data(housing_data)

In [4]:
# 5-Fold and 10-Fold Cross-validation
def k_fold_cross_validation(X, y, k_folds, hidden_size, learning_rate):
    fold_size = len(X) // k_folds
    fold_results = []
    
    for i in range(k_folds):
        # Split the data into training and validation sets
        X_train = np.concatenate((X[:i * fold_size], X[(i + 1) * fold_size:]), axis=0)
        y_train = np.concatenate((y[:i * fold_size], y[(i + 1) * fold_size:]), axis=0)
        X_val = X[i * fold_size: (i + 1) * fold_size]
        y_val = y[i * fold_size: (i + 1) * fold_size]
        
        # Initialize the neural network
        input_size = X.shape[1]
        output_size = 1  # Predicting a single output (regression)
        nn = NeuralNetwork(input_size, hidden_size, output_size, learning_rate)
        
        # Train the model
        nn.train(X_train, y_train, epochs=1000)
        
        # Test on the validation set
        predictions = nn.predict(X_val)
        loss = mse_loss(y_val, predictions)
        acc = accuracy(y_val, predictions)
        print(f"Fold {i+1}, Loss: {loss}, Accuracy (R²): {acc}")
        
        fold_results.append((loss, acc))
    
    # Average results across all folds
    avg_loss = np.mean([result[0] for result in fold_results])
    avg_acc = np.mean([result[1] for result in fold_results])
    print(f"\nAverage Loss: {avg_loss}, Average Accuracy (R²): {avg_acc}")

# User inputs for the neural network configuration
hidden_layer_size = int(input("Enter the number of neurons in the hidden layer: "))
learning_rate = float(input("Enter the learning rate: "))
output_layer_size = 1  # Since we're doing regression

# Running the experiments with different configurations
print(f"5-Fold Cross-Validation for Case (a1): Hidden Layer Size = {hidden_layer_size}, Learning Rate = {learning_rate}")
k_fold_cross_validation(X, y, k_folds=5, hidden_size=hidden_layer_size, learning_rate=learning_rate)

print(f"\n10-Fold Cross-Validation for Case (a2): Hidden Layer Size = {hidden_layer_size}, Learning Rate = {learning_rate}")
k_fold_cross_validation(X, y, k_folds=10, hidden_size=hidden_layer_size, learning_rate=learning_rate)

# User inputs for the neural network configuration
hidden_layer_size = int(input("Enter the number of neurons in the hidden layer: "))
learning_rate = float(input("Enter the learning rate: "))
output_layer_size = 1  # Since we're doing regression

# Running the experiments with different configurations
print(f"5-Fold Cross-Validation for Case (b1): Hidden Layer Size = {hidden_layer_size}, Learning Rate = {learning_rate}")
k_fold_cross_validation(X, y, k_folds=5, hidden_size=hidden_layer_size, learning_rate=learning_rate)

print(f"\n10-Fold Cross-Validation for Case (b2): Hidden Layer Size = {hidden_layer_size}, Learning Rate = {learning_rate}")
k_fold_cross_validation(X, y, k_folds=10, hidden_size=hidden_layer_size, learning_rate=learning_rate)

# User inputs for the neural network configuration
hidden_layer_size = int(input("Enter the number of neurons in the hidden layer: "))
learning_rate = float(input("Enter the learning rate: "))
output_layer_size = 1  # Since we're doing regression

# Running the experiments with different configurations
print(f"\n5-Fold Cross-Validation for Case (c1): Hidden Layer Size = {hidden_layer_size}, Learning Rate = {learning_rate}")
k_fold_cross_validation(X, y, k_folds=5, hidden_size=hidden_layer_size, learning_rate=learning_rate)

print(f"\n10-Fold Cross-Validation for Case (c2): Hidden Layer Size = {hidden_layer_size}, Learning Rate = {learning_rate}")
k_fold_cross_validation(X, y, k_folds=10, hidden_size=hidden_layer_size, learning_rate=learning_rate)

Enter the number of neurons in the hidden layer:  3
Enter the learning rate:  0.01


5-Fold Cross-Validation for Case (a1): Hidden Layer Size = 3, Learning Rate = 0.01
Epoch 0, Loss: 0.1392008784218467
Epoch 100, Loss: 0.011450801917266926
Epoch 200, Loss: 0.010480518019733332
Epoch 300, Loss: 0.009998842048308507
Epoch 400, Loss: 0.009628150566287014
Epoch 500, Loss: 0.009303928649552288
Epoch 600, Loss: 0.009011463957115802
Epoch 700, Loss: 0.008742649212008178
Epoch 800, Loss: 0.00848178332625007
Epoch 900, Loss: 0.008214096184964567
Fold 1, Loss: 0.0027605115341776034, Accuracy (R²): 0.8028232361550633
Epoch 0, Loss: 0.09684638403228671
Epoch 100, Loss: 0.009446259701537549
Epoch 200, Loss: 0.008374181658019118
Epoch 300, Loss: 0.007616741181726217
Epoch 400, Loss: 0.0070375260024347274
Epoch 500, Loss: 0.006584661948561024
Epoch 600, Loss: 0.0062232147119671115
Epoch 700, Loss: 0.005934031851729138
Epoch 800, Loss: 0.005704359106524796
Epoch 900, Loss: 0.005523093682631494
Fold 2, Loss: 0.006777544084627674, Accuracy (R²): 0.7650681667648855
Epoch 0, Loss: 0.08745

Enter the number of neurons in the hidden layer:  4
Enter the learning rate:  0.001


5-Fold Cross-Validation for Case (b1): Hidden Layer Size = 4, Learning Rate = 0.001
Epoch 0, Loss: 0.059672337306777616
Epoch 100, Loss: 0.036296228454533716
Epoch 200, Loss: 0.024410389321619312
Epoch 300, Loss: 0.018837177036131216
Epoch 400, Loss: 0.015798586211378727
Epoch 500, Loss: 0.013994725834996346
Epoch 600, Loss: 0.012861013696236033
Epoch 700, Loss: 0.012109836770041368
Epoch 800, Loss: 0.011586631715251473
Epoch 900, Loss: 0.011204845618790092
Fold 1, Loss: 0.004650058380023308, Accuracy (R²): 0.6678574055166275
Epoch 0, Loss: 0.04197117680479531
Epoch 100, Loss: 0.01572779751588371
Epoch 200, Loss: 0.013065857045445982
Epoch 300, Loss: 0.01189210243434245
Epoch 400, Loss: 0.011361295264799771
Epoch 500, Loss: 0.011083225028331279
Epoch 600, Loss: 0.010898775376871369
Epoch 700, Loss: 0.010750208449279366
Epoch 800, Loss: 0.010617580879812563
Epoch 900, Loss: 0.010493958580846062
Fold 2, Loss: 0.011480898529428858, Accuracy (R²): 0.6020345268099833
Epoch 0, Loss: 0.184078

Enter the number of neurons in the hidden layer:  5
Enter the learning rate:  0.0001



5-Fold Cross-Validation for Case (c1): Hidden Layer Size = 5, Learning Rate = 0.0001
Epoch 0, Loss: 0.0749547312558926
Epoch 100, Loss: 0.05838009412967923
Epoch 200, Loss: 0.042971711865112554
Epoch 300, Loss: 0.030728584454315255
Epoch 400, Loss: 0.022473363184308343
Epoch 500, Loss: 0.01765465442197903
Epoch 600, Loss: 0.015122169077497149
Epoch 700, Loss: 0.013863537170802197
Epoch 800, Loss: 0.013237509391958543
Epoch 900, Loss: 0.012904740680135841
Fold 1, Loss: 0.004016207769922755, Accuracy (R²): 0.7131318448780987
Epoch 0, Loss: 0.05642940979297587
Epoch 100, Loss: 0.04481977862092068
Epoch 200, Loss: 0.03711821573397541
Epoch 300, Loss: 0.032207878982818544
Epoch 400, Loss: 0.028846390182576884
Epoch 500, Loss: 0.026290056437074202
Epoch 600, Loss: 0.02419381500767348
Epoch 700, Loss: 0.022408543721526678
Epoch 800, Loss: 0.02086409009882582
Epoch 900, Loss: 0.019520042423185568
Fold 2, Loss: 0.015609020996347023, Accuracy (R²): 0.4589403075968853
Epoch 0, Loss: 0.0904833150